In [30]:
import os
import sys
from time import sleep
from uuid import uuid4

import GCode
import GRBL
import numpy as np

# Laser Circles 

# Code:

In [2]:
cnc = GRBL.GRBL(port="/dev/cnc_3018")
sleep(0.5)
cnc.laser_mode = 1
sleep(0.5)
print("Laser Mode: {}".format(cnc.laser_mode))
cnc.home()


ok
Laser Mode: 1.0


10

In [3]:
def init():
    program = GCode.GCode()
    program.G0(F=500)
    program.G1(F=300)
    program.M3(S=1)
    program.G28()
    program.G21()  # Metric Units
    program.G90()  # Absolute positioning.
    program.G92(X=0, Y=0)
    program.M5()
    return program


def end():
    program = GCode.GCode()
    program.M5()
    program.G28()
    return program


In [12]:
def circle(center_x=10, center_y=10, radius=10, laser_pwm=255, clockwise=True):
    prog = GCode.GCode()
    prog.G0(X=(center_x - radius), Y=(center_y))
    prog.M3(S=laser_pwm)
    arc_cfg={
        "X": (center_x - radius),
        "Y": (center_y),
        "I": radius,
        "J": 0,      
    }
    if clockwise:
        prog.G2(**arc_cfg)
    else:
        prog.G3(**arc_cfg)
    prog.M5()
    return prog


In [7]:

circle()


<GCode>[cmds=4]

In [10]:
radius = 20


In [19]:
# Poplar 1x4". Cut
BlockHeight = 89.0  # mm
BlockLength = 2 * BlockHeight  # mm


In [20]:
BlockHeight / 2


44.5

In [23]:
circle_program = GCode.GCode()
circle_program += init()
# Default circle, Center at 10mm,10mm radius 10mm
circle_program += circle()
# 25mm
circle_program += circle(
    center_x=25,
    center_y=25,
    radius=25,
    laser_pwm=255,
)
# Block Height
circle_program += circle(
    center_x=BlockHeight / 2,
    center_y=BlockHeight / 2,
    radius=BlockHeight /2,
    laser_pwm=255,
)
circle_program += end()

In [24]:
circle_program

<GCode>[cmds=22]

In [26]:
circle_program.machine = cnc


In [31]:
circle_program.run()


In [28]:
BlockHeight

89.0

In [29]:
BlockLength

178.0